In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import integrate
from scipy.integrate import solve_ivp

In [2]:
plt.rc('text', usetex = True)
plt.rc('font', **{'family' : "sans-serif"})
params = {'text.latex.preamble' : [r'\usepackage{amsmath}']}
plt.rcParams.update(params)
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8

In [3]:
def funciones(xi,y, sigma, n, h):
    psi_,eta_,psi1_,eta1_ = y
    dydxi = [h*(eta_ + sigma*xi**(3)*psi_**(n+1))*((1 - n*sigma) + (n+1)*sigma*psi_)/xi/(2*(n+1)*sigma*eta_ - xi)
           ,xi**(2)*psi_**(n)*(1 - n*sigma + n*sigma*psi_) 
           ,h*(((eta1_ + 3*sigma*xi**(2)*psi_**(n+1) + sigma*xi**(3)*(n+1)*psi_**(n)*psi1_)*((1 - n*sigma) + (n + 1)*sigma*psi_)/xi/(2*(n+1)*sigma*eta_ - xi)) + (sigma*(n+1)*psi1_)*(eta_ + sigma*xi**(3)*psi_**(n+1))/xi/(2*(n+1)*sigma*eta_ - xi)  -  (((1 - n*sigma) + (n + 1)*sigma*psi_)*(eta_ + sigma*xi**(3)*psi_**(n+1))/xi**(2)/(2*(n+1)*sigma*eta_ - xi) )  -  (((1 - n*sigma) + (n + 1)*sigma*psi_)*(eta_ + sigma*xi**(3)*psi_**(n+1))*(2*(n+1)*sigma*eta1_ - 1)/xi/((2*(n+1)*sigma*eta_ - xi)**(2))))         
           ,(2*xi*psi_**(n) + n*xi**(2)*psi_**(n-1)*psi1_)*(1 - n*sigma + n*sigma* psi_) + xi**(2)*psi_**(n)*n*sigma*psi1_]
    return dydxi

In [4]:
c = 2.997*10**(8)               # Velocidad de la luz en S.I.
G = 6.673*10**(-11)             # Constante Gravitación Universal en S.I. 
Msun = 1.989*10**(30)           # 1 Masa solar en S.I.

In [5]:
                                               ## Lista de parámetros ##

# C
C_inicial = 0                    # C inicial            
C_final =  1/4                   # C final
Paso_C = 1/32                    # Diferencia entre valores consecutivos
Num_C = round((Paso_C + C_final - C_inicial)/Paso_C)  # Número de valores
Lista_C = np.linspace(C_inicial,C_final,Num_C)        # Lista con valores desde C inicial hasta C final
Lista_C = [0, 1/16, 1/8, 3/16, 1/4]
print('C interval: ',end='')
print(*Lista_C, sep=', ')

# n 
n_inicial = 0.5          # n inicial 
n_final = 0.4             # n final
Paso_n = 0.5             # Diferencia entre valores consecutivos
Num_n = round((Paso_n + n_final - n_inicial)/Paso_n)  # Número de valores
Lista_n = np.linspace(n_inicial,n_final,Num_n)        # Lista con valores desde n inical hasta n final
Lista_n = [0.5,1.5,3.0]
print('n interval: ',end='')                                                    
print(*Lista_n, sep=', ')

# Sigma = razón entre Presión central y Densidad central
sigma_inicial = 0.05             # sigma inicial             
sigma_final = 0.8                # sigma final
Paso_sigma = 0.05                # Diferencia entre valores consecutivos
Num_sigma = round((Paso_sigma + sigma_final - sigma_inicial)/Paso_sigma)  # Número de valores
Lista_sigma = np.linspace(sigma_inicial,sigma_final,Num_sigma)            # Lista con valores desde sigma inical hasta sigma final
Lista_sigma = [0.2,0.4,0.6,0.8]
print('\u03c3 interval: ',end='')
print(*Lista_sigma, sep=', ')

Number_of_models = len(Lista_C)*len(Lista_n)*len(Lista_sigma)
print('Number of models: ', Number_of_models)

rho_c = 2.5*10**(18)               # Densidad central

C interval: 0, 0.0625, 0.125, 0.1875, 0.25
n interval: 0.5, 1.5, 3.0
σ interval: 0.2, 0.4, 0.6, 0.8
Number of models:  60


In [6]:
ListaModelos = []                                        # Container for all models 

for i in range(len(Lista_C)):
    
    ListaModelos.append([])                              # Container for C
    
    for j in range(len(Lista_n)):
        
        ListaModelos[i].append([])                       # Container for n

In [7]:
%%time

contador = 0

for i in range(len(Lista_C)):
    
    C = Lista_C[i]
    h = 1 - 2*C
    
    for j in range(len(Lista_n)):
        
        n = Lista_n[j]
        
        for k in range(len(Lista_sigma)):
            
            sigma = Lista_sigma[k]
                    
            K = sigma*c**(2)/rho_c**(1/n)      # k**(n) en m**(3)/kg
                     
            # Condiciones iniciales
            Psi0 = 1.0
            Eta0 = 0.0
            Psi10 = 0.0
            Eta10 = 0.0
            
            y0 = [Psi0,Eta0,Psi10,Eta10]
            
            def stop_condition(xi,y,sigma,n,h):
                return y[0] - 10**-14
            
            stop_condition.terminal = True
            
            xi0 = 10**(-14)
            ximax = 10000
    
            xi_span = (xi0,ximax)
        
            soluciones = solve_ivp(funciones,xi_span,y0,method='RK45',events=stop_condition,
                                   args=(sigma,n,h))
            
            if soluciones.status == 0:
                print ("insuficiente intervalo de integración xi por favor auméntelo")
                contador = contador + 1
                continue
            else:
                Psi = soluciones.y[0]
                Eta = soluciones.y[1]
                Psi1 = soluciones.y[2]
                Eta1 = soluciones.y[3]
                xi = soluciones.t
            
            #########################################################################################################
                
            xiNorm = xi/xi[-1]                                       # xi normalized
            
            PNorm = Psi**(n+1)
            
            m = (c**(2)*sigma*(n+1)/(4*np.pi*rho_c)**(1/3)/G)**(3/2)*Eta/Msun   # Masa en unidades de masa solar
            r = (c**(2)*sigma*(n+1)/4/np.pi/G/rho_c)**(1/2)*xi/1000             # Radio en kilómetros
            mr = 2*sigma*(n+1)*Eta/xi                                           # 2*G*m/c**(2)/r
            
            Densidad = Psi**(n)                                      # Density
            
            DensidadEnergia = (1 - n*sigma)*Psi**(n) + n*sigma*Psi**(n+1)
            
            Dprima = n*Psi**(n-1)*Psi1                               # Density gradient
            
            # Tangential pressure
            PTNorm = (C*(n+1)*(Eta + sigma*Psi**(n+1)*xi**(3))/(xi - 2*Eta*sigma*(n+1)))*((1 - n*sigma)*Psi**(n) + (n+1)*sigma*Psi**(n+1)) + Psi**(n+1)
                    
            Pprima = (n+1)*Psi**(n)*Psi1
            
            # Tangential pressure gradient
            Ptprima = C*(n+1)*((Eta1 + 3*sigma*xi**(2)*Psi**(n+1) + sigma*(n+1)*xi**(3)*Psi**(n)*Psi1)*((1-sigma*n)*Psi**(n) + sigma*(n+1)*Psi**(n+1))/(xi - 2*Eta*(n+1)*sigma) + (Eta + sigma*xi**(3)*Psi**(n+1))*(n*(1 - sigma*n)*Psi**(n-1)*Psi1 + sigma*(n+1)**(2)*Psi**(n)*Psi1)/(xi - 2*Eta*(n+1)*sigma) - (Eta + sigma*xi**(3)*Psi**(n+1))*((1 - sigma*n)*Psi**(n) + sigma*(n+1)*Psi**(n+1))*(1 - 2*Eta1*(n+1)*sigma)/(xi - 2*Eta*(n+1)*sigma)**(2)) + (n+1)*Psi**(n)*Psi1
            
            SEC = (1/sigma)*DensidadEnergia - PNorm - 2*PTNorm                # Strong energy condition (SEC)   
            
            v2r =  ((n+1)/n)*(sigma*Psi)/(sigma*Psi*(n+1) + (1 - n*sigma))                      # Sound speed squared
            
            # Tangential sound speed squared
            v2t = C*sigma*(n+1)*(Eta + sigma*Psi**(n+1)*xi**(3))/(xi - 2*Eta*sigma*(n+1)) + ((n+1)/n)*(sigma*Psi)/(sigma*Psi*(n+1) + (1 - n*sigma))
            
            Gamma = v2r*(DensidadEnergia + sigma*PNorm)/PNorm/sigma
            
            # Second derivative of Psi
            Psi11 = h*(((Eta1 + 3*sigma*xi**(2)*Psi**(n+1) + sigma*xi**(3)*(n+1)*Psi**(n)*Psi1)*((1 - n*sigma) + (n+1)*sigma*Psi)/xi/(2*(n+1)*sigma*Eta - xi)) + (sigma*(n+1)*Psi1)*(Eta + sigma*xi**(3)*Psi**(n+1))/xi/(2*(n+1)*sigma*Eta - xi) - (((1 - n*sigma) + (n+1)*sigma*Psi)*(Eta + sigma*xi**(3)*Psi**(n+1))/xi**(2)/(2*(n+1)*sigma*Eta - xi))     -     (((1 - n*sigma) + (n+1)*sigma*Psi)*(Eta + sigma*xi**(3)*Psi**(n+1))*(2*(n+1)*sigma*Eta1 - 1)/xi/((2*(n+1)*sigma*Eta - xi)**(2))))
            
            # Cracking: deltaR1: Density, deltaR2: Mass, deltaR3: Pressure, delta R4: Pressure gradient   
            deltaR1 = h*((n+1)*sigma*(Eta + xi**(3)*sigma*Psi**(n+1))/xi/(xi - 2*sigma*n*Eta - 2*sigma* Eta))
            deltaR2 = h*(((1 - n*sigma)*Psi**(n) + (n+1)*sigma*Psi**(n+1))*(1 + 2*xi**(2)*sigma**(2)*(n+1)*Psi**(n+1))/(xi - 2*sigma*(n+1)*Eta)**(2))*(xi**(2)*sigma*(n+1)*((1 - n*sigma)*Psi**(n) + n*sigma*Psi**(n+1))/((1 - n*sigma)*n*Psi**(n-1)*Psi1 + n*sigma*(n+1)*Psi**(n)*Psi1))
            deltaR3 = h*(sigma*(n+1)*(Eta + xi**(3)*(1 - n*sigma)*Psi**(n) + xi**(3)*sigma*(n+2)*Psi**(n+1))/xi/(xi - 2*sigma*(n+1)*Eta))*(1 + 1/n)*(sigma*Psi/(sigma*Psi*(n+1) + (1 - n*sigma)))
            deltaR4 = sigma*(n+1)*(Psi**(n)*Psi11 + n*Psi**(n-1)*Psi1**(2))/((1 - n*sigma)*n*Psi**(n-1)*Psi1 + n*sigma*(n+1)*Psi**(n)*Psi1)
                      
            deltaRPolitropa = deltaR1 + deltaR2 + deltaR3 + deltaR4
            
            # Buoyancy
            Psin11 = (n*(1 - n*sigma))*(Psi**(n-1)*Psi11 + (n-1)*Psi**(n-2)*Psi1**(2)) + (n*sigma*(n+1))*(Psi**(n)*Psi11 + n*Psi**(n-1)*Psi1**(2))
            
            M = (c**(2)*(n+1)/(4*np.pi)**(1/3)/G)**(3/2)*(k/c**(2))**(n/2)*sigma**((3-n)/2)*Eta[-1]/Msun  # Masa en unidades de masa solar
            
            R = (c**(2)*(n+1)/4/np.pi/G)**(1/2)*(k/c**(2))**(n/2)*sigma**((1-n)/2)*xi[-1]/1000         # Radio en kilómetros
            
            ##########################################################################################################
            
                                         #0    #1     #2       #3     #4      #5      #6      #7                       
            ListaModelos[i][j].append([xiNorm, mr, Densidad, PNorm, PTNorm, Dprima, Pprima, Ptprima,
                                        #8  #9   #10   #11          #12         #13   #14  #15   #16         #17
                                       SEC, v2r, v2t, Gamma, deltaRPolitropa, Psin11,  C  , n , sigma, DensidadEnergia])
        


C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.
C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  
C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:89: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:91: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:93: RuntimeWarning: invalid val

insuficiente intervalo de integración xi por favor auméntelo
insuficiente intervalo de integración xi por favor auméntelo
insuficiente intervalo de integración xi por favor auméntelo
insuficiente intervalo de integración xi por favor auméntelo
insuficiente intervalo de integración xi por favor auméntelo
insuficiente intervalo de integración xi por favor auméntelo
insuficiente intervalo de integración xi por favor auméntelo
insuficiente intervalo de integración xi por favor auméntelo
insuficiente intervalo de integración xi por favor auméntelo
insuficiente intervalo de integración xi por favor auméntelo
insuficiente intervalo de integración xi por favor auméntelo
insuficiente intervalo de integración xi por favor auméntelo
insuficiente intervalo de integración xi por favor auméntelo
insuficiente intervalo de integración xi por favor auméntelo
insuficiente intervalo de integración xi por favor auméntelo
insuficiente intervalo de integración xi por favor auméntelo
insuficiente intervalo d

In [8]:
print(contador, ' de ', Number_of_models, ' NO son integrables')

20  de  60  NO son integrables


# <center> 2m/r < 1
# <center> C1, C2 and C3

In [10]:
count = 0

for i in range(len(Lista_C)):
    
    for j in range(len(Lista_n)):
        
        for k in range(len(Lista_sigma)):
            
            if len(ListaModelos[i][j][k][1]) != 0:
                
                if any(ListaModelos[i][j][k][1] >= 1):
                    
                    print('2m/r > 1. Parámetros: ','C = %.3f, '%(ListaModelos[i][j][k][14]),
                          'n = %.1f, '%(ListaModelos[i][j][k][15]),'\u03c3 = %.2f, '%(ListaModelos[i][j][k][16]))
                    count = count + 1
                
                else:
                    continue
            else:
                continue
                
print('Número total de modelos que NO cumplen 2m/r < 1:           ', count, ' de ', contador)


IndexError: list index out of range

# <center> Density, Radial Pressure and Tangential Pressure   > 0
# <center> C4

In [11]:
count1 = 0
count2 = 0
count3 = 0

for i in range(len(Lista_C)):
    
    for j in range(len(Lista_n)):
        
        for k in range(len(Lista_sigma)):
            
            if len(ListaModelos[i][j][k]) != 0:
                
                if any(ListaModelos[i][j][k][2] < 0):
                    
                    print('Densidad < 0. Parámetros: ','C = %.3f, '%(ListaModelos[i][j][k][14]),
                         'n = %.1f, '%(ListaModelos[i][j][k][15]),'\u03c3 = %.2f, '%(ListaModelos[i][j][k][16]))
                    count1 = count1 + 1
                else:
                    pass
                
                if any(ListaModelos[i][j][k][3] < 0):
                    
                    print('Presión radial < 0. Parámetros: ','C = %.3f, '%(ListaModelos[i][j][k][14]),
                         'n = %.1f, '%(ListaModelos[i][j][k][15]),'\u03c3 = %.2f, '%(ListaModelos[i][j][k][16]))
                    count2 = count2 + 1
                    
                else:
                    pass
                
                if any(ListaModelos[i][j][k][4] < 0):
                    
                    print('Presión tangencial < 0. Parámetros: ','C = %.3f, '%(ListaModelos[i][j][k][14]),
                         'n = %.1f, '%(ListaModelos[i][j][k][15]),'\u03c3 = %.2f, '%(ListaModelos[i][j][k][16]))
                    count3 = count3 + 1
                else:
                    continue
                
print('Número total de modelos que no cumplen Densidad > 0:           ', count1, ' de ', contador)
print('Número total de modelos que no cumplen Presión radial > 0:     ', count2, ' de ', contador)
print('Número total de modelos que no cumplen Presión tangencial > 0: ', count3, ' de ', contador)

IndexError: list index out of range

# <center> Density gradient, Radial Pressure gradient and Tangetial Pressure gradient  < 0
# <center> C5

In [ ]:
count1 = 0
count2 = 0
count3 = 0

for i in range(len(Lista_C)):
    
    for j in range(len(Lista_n)):
        
        for k in range(len(Lista_sigma)):
            
            if len(ListaModelos[i][j][k]) != 0:
            
                if any(ListaModelos[i][j][k][5][50:] > 0):
                    
                    print('Gradiente de densidad > 0. Parámetros: ','C = %.3f, '%(ListaModelos[i][j][k][14]),
                         'n = %.1f, '%(ListaModelos[i][j][k][15]),'\u03c3 = %.2f, '%(ListaModelos[i][j][k][16]))
                    count1 = count1 + 1
                else:
                    pass
                
                if any(ListaModelos[i][j][k][6][50:] > 0):
                    
                    print('Gradiente de presión > 0. Parámetros: ','C = %.3f, '%(ListaModelos[i][j][k][14]),
                         'n = %.1f, '%(ListaModelos[i][j][k][15]),'\u03c3 = %.2f, '%(ListaModelos[i][j][k][16]))
                    count2 = count2 + 1
                else:
                    pass
                
                if any(ListaModelos[i][j][k][7][2000:] > 0):
                    
                    print('Gradiente de presión tangencial > 0. Parámetros: ','C = %.3f, '%(ListaModelos[i][j][k][14]),
                         'n = %.1f, '%(ListaModelos[i][j][k][15]),'\u03c3 = %.2f, '%(ListaModelos[i][j][k][16]))
                    count3 = count3 + 1
                else:
                    continue

print('Número total de modelos que no cumplen Gradiende de Densidad < 0:           ', count1, ' de ',contador)
print('Número total de modelos que no cumplen Gradiente de Presión radial < 0:     ', count2, ' de ',contador)
print('Número total de modelos que no cumplen Gradiente de Presión tangencial < 0: ', count3, ' de ',contador)


# <center> Strong Energy Condition
# <center> C6

In [ ]:
count = 0

for i in range(len(Lista_C)):
    
    for j in range(len(Lista_n)):
        
        for k in range(len(Lista_sigma)):
            
            if len(ListaModelos[i][j][k]) != 0:
            
                if any(ListaModelos[i][j][k][8] < 0):
                                        
                    print('No cumple la condición de energía fuerte (SEC). Parámetros: ','C = %.3f, '%(ListaModelos[i][j][k][14]),
                         'n = %.1f, '%(ListaModelos[i][j][k][15]),'\u03c3 = %.2f, '%(ListaModelos[i][j][k][16]))
                    count = count + 1
                else:
                    continue

print('Número total de modelos que NO cumplen: ', count, ' de ',contador)

# <center> Radial and Tangential speed of sound squared
# <center> C7

In [ ]:
count1 = 0
count2 = 0

for i in range(len(Lista_C)):
    
    for j in range(len(Lista_n)):
        
        for k in range(len(Lista_sigma)):
            
            if len(ListaModelos[i][j][k]) != 0:
            
                if any(ListaModelos[i][j][k][9] > 1):
                    print('Velocidad del sonido > 1.            Parámetros: ','C = %.3f, '%(ListaModelos[i][j][k][14]),
                         'n = %.1f, '%(ListaModelos[i][j][k][15]),'\u03c3 = %.2f, '%(ListaModelos[i][j][k][16]))
                    count1 = count1 + 1
                else:
                    pass
                
                if any(ListaModelos[i][j][k][10] > 1):                
                    print('Velocidad del sonido tangencial > 1. Parámetros: ','C = %.3f, '%(ListaModelos[i][j][k][14]),
                         'n = %.1f, '%(ListaModelos[i][j][k][15]),'\u03c3 = %.2f, '%(ListaModelos[i][j][k][16]))
                    count2 = count2 + 1
                else:
                    continue
                
print('Número total de modelos que no cumplen: ', count1, ' de ',contador)
print('Número total de modelos que no cumplen: ', count2, ' de ',contador)

# <center> Adiabatic index Gamma
# <center> C8

In [ ]:
count = 0

for i in range(len(Lista_C)):
    
    for j in range(len(Lista_n)):
        
        for k in range(len(Lista_sigma)):
            
            if len(ListaModelos[i][j][k]) != 0:
                
                if any(ListaModelos[i][j][k][11] < 1.33333):
                    print('Gamma < 4/3. Parámetros: ','C = %.3f, '%(ListaModelos[i][j][k][14]),
                         'n = %.1f, '%(ListaModelos[i][j][k][15]),'\u03c3 = %.2f, '%(ListaModelos[i][j][k][16]))
                    count = count + 1
                    print(ListaModelos[i][j][k][11])
                else:
                    continue
                
print('Número total de modelos que no cumplen: ', count, ' de ',contador)

# <center> Cracking
# <center> C9

In [ ]:
count = 0

for i in range(len(Lista_C)):
    
    for j in range(len(Lista_n)):
        
        for k in range(len(Lista_sigma)):
            
            if len(ListaModelos[i][j][k]) != 0:
            
                if any(np.diff(np.sign(ListaModelos[i][j][k][12][100:])) != 0):
                    print('El modelo fractura. Parámetros: ','C = %.3f, '%(ListaModelos[i][j][k][14]),
                         'n = %.1f, '%(ListaModelos[i][j][k][15]),'\u03c3 = %.2f, '%(ListaModelos[i][j][k][16]))
                    count = count + 1
                else:
                    continue
                
print('Número total de modelos que NO cumplen: ', count, ' de ',contador)

# <center> Convective Stability
# <center> C11

In [ ]:
count = 0

for i in range(len(Lista_C)):
    
    for j in range(len(Lista_n)):
        
        for k in range(len(Lista_sigma)):
            
            if len(ListaModelos[i][j][k]) != 0:
            
                if any(ListaModelos[i][j][k][13] > 0):
                    print('Inestable ante perturbaciones convectivas. Parámetros: ','C = %.3f, '%(ListaModelos[i][j][k][14]),
                         'n = %.1f, '%(ListaModelos[i][j][k][15]),'\u03c3 = %.2f, '%(ListaModelos[i][j][k][16]))
                    count = count + 1                
                else:
                    continue
                
print('Número total de modelos que NO cumplen: ', count, ' de ',contador)